In [318]:
import pandas as pd
import numpy as np
import json

In [319]:
myFile = pd.read_csv('D:/project/550_Pattern/Project_1/550-p1-cset-krk-1.csv')
myFile
myFrame=pd.DataFrame(myFile.values, columns = ["White-King-File", "White-King-rank", "White-Rook-file", "White-Rook-rank","Black-King-file","Black-King-rank","Class"])
myFrame

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,1,e,2,d,3,draw
1,b,1,d,7,c,7,draw
2,d,2,g,1,g,2,draw
3,d,4,g,6,f,6,draw
4,c,1,b,5,a,5,draw
...,...,...,...,...,...,...,...
214,d,3,d,7,a,7,nine
215,b,1,b,4,g,1,nine
216,d,4,e,8,g,2,nine
217,c,2,d,5,f,1,nine


In [320]:
myFrame.shape[0]

219

In [321]:
# Split into training set:60%, holdout set:20%, Validation set: 20%

training_set = myFrame.sample(frac=0.6)
remaining_set = myFrame.drop(training_set.index)
training_set.reset_index(drop=True, inplace=True)

In [322]:
training_set

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,d,3,f,4,e,1,one
1,d,3,a,4,a,2,six
2,c,2,f,2,a,2,two
3,d,3,b,5,h,1,six
4,c,2,h,3,g,3,draw
...,...,...,...,...,...,...,...
126,a,1,a,2,g,1,seven
127,c,2,a,8,a,2,zero
128,d,3,f,6,f,7,draw
129,d,3,b,5,a,2,four


In [323]:
remaining_set.reset_index(drop=True, inplace=True)
remaining_set

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,d,2,g,1,g,2,draw
1,d,4,g,6,f,6,draw
2,c,1,b,5,a,5,draw
3,c,1,g,2,a,1,draw
4,d,1,g,3,h,4,draw
...,...,...,...,...,...,...,...
83,d,3,h,1,f,1,nine
84,c,1,a,5,f,1,nine
85,c,3,c,7,h,2,nine
86,d,3,d,7,a,7,nine


In [324]:
classes, cls_counts = np.unique(training_set['Class'], return_counts = True)
classes, cls_counts


(array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
        'three', 'two', 'zero'], dtype=object),
 array([11, 13, 10, 13, 16, 10, 12, 15,  9, 15,  7]))

In [325]:
def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
                        for i in range(len(classes))])
    return entropy_value

In [326]:
calculate_entropy(training_set['Class'])

3.4218900265853938

In [327]:

for cols in training_set.columns[:-1]:
    print(cols)

White-King-File
White-King-rank
White-Rook-file
White-Rook-rank
Black-King-file
Black-King-rank


In [328]:
# dataset = training_set
# feature = Feature or attribute of the dataset, i.e column name (training_set.columns[:-1]) E.g White-King-File
# label = Name of Class label colums, i.e. 'Class'

def calculate_information_gain(dataset,feature,label): 
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])   
    values,feat_counts= np.unique(dataset[feature],return_counts=True)
    
    # Calculate the weighted feature entropy                                # Call the calculate_entropy function
    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]
                              ==values[i]).dropna()[label]) for i in range(len(values))])    
    feature_info_gain = dataset_entropy - weighted_feature_entropy    
    return feature_info_gain

In [329]:
calculate_information_gain(training_set, 'White-King-File', 'Class')

0.47708421236098797

In [330]:
class DNode:
    def __init__(self, attribute, values):
        self.attribute = attribute
        self.values = values
        self.children = {}
    

In [331]:
class LeafNode:
    def __init__(self,value):
        self.value = value
        

In [332]:
def copy_list(l):
    return l[:]

In [333]:
attributes = training_set.columns[:-1].tolist()
class_label = 'Class'

In [334]:
def select_attribute(dataset, attributes, class_label):
    
    gains = [calculate_information_gain(dataset, attribute, class_label) for attribute in attributes]
#     print("gains",gains)
    maxGainIndex = np.argmax(gains)
    maxGainValue = gains[maxGainIndex]
#     print("maxGainIndex",maxGainIndex)
#     print("maxGainValue",maxGainValue)
#     print("highGainAttribute:",attributes[maxGainIndex])
    
    
    
    if maxGainValue == 0.0:
        value = np.unique(dataset[class_label])[0]
#         print('value',value)
        return LeafNode(value)
    
    else:
        newAttribute = attributes[maxGainIndex]
        values = np.unique(dataset[newAttribute])
#         print("newAttribute:",newAttribute)
#         print("values:",values)
        return DNode(newAttribute, values)
        

In [335]:
select_attribute(training_set, attributes, class_label)

In [336]:
def create_dtree(dataset, attributes, class_label):        
    node = select_attribute(dataset, attributes, class_label)
    
    if isinstance(node, DNode):
        newDataSet = dataset.drop(columns = node.attribute)        
        newAttributes = copy_list(attributes)
        newAttributes.remove(node.attribute)        
#         print('newAttributes', newAttributes)
#         print('node.values',node.values)
        for v in node.values:
#             print("v:",v)
            node.children[v] = create_dtree(dataset.where(dataset[node.attribute] == v).dropna(), newAttributes, class_label)
    
    return node

In [337]:
rootNode = create_dtree(training_set, copy_list(attributes), class_label)

In [338]:
def check_treeRec(node):
    if isinstance(node, LeafNode):
        print('leaf node value:', node.value)
        return 
    
    for k,v in node.children.items():
        check_treeRec(v)
    
    return

In [339]:
check_treeRec(rootNode)

leaf node value: zero
leaf node value: six
leaf node value: two
leaf node value: three
leaf node value: one
leaf node value: three
leaf node value: three
leaf node value: six
leaf node value: four
leaf node value: eight
leaf node value: seven
leaf node value: two
leaf node value: six
leaf node value: one
leaf node value: two
leaf node value: four
leaf node value: four
leaf node value: one
leaf node value: five
leaf node value: eight
leaf node value: two
leaf node value: four
leaf node value: three
leaf node value: two
leaf node value: four
leaf node value: two
leaf node value: five
leaf node value: six
leaf node value: one
leaf node value: three
leaf node value: nine
leaf node value: eight
leaf node value: three
leaf node value: four
leaf node value: five
leaf node value: four
leaf node value: two
leaf node value: eight
leaf node value: two
leaf node value: five
leaf node value: four
leaf node value: four
leaf node value: eight
leaf node value: four
leaf node value: three
leaf node val

In [352]:
def predict(node, row, correctAns):
    if isinstance(node, LeafNode):
        return node.value == correctAns
#     print('Attr:',node.attribute,", value:", row[node.attribute])
    if row[node.attribute] not in node.children:
        return False
    return predict(node.children[row[node.attribute]], row, correctAns)
    

In [353]:
# ans = predict(rootNode, training_set.loc[126], 'draw')


In [354]:
count = 0
for i in range(remaining_set.shape[0]):
    ans = predict(rootNode, remaining_set.loc[i], remaining_set.loc[i][-1])
    if ans:
        count+=1
    print(ans)
print(count)

False
True
False
False
False
False
True
False
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
True
True
False
True
True
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
True
False
False
False
False
False
False
False
False
True
False
False
False
False
True
False
False
False
17
